# Data queries - Chemistry

Chemistry is one of the focal domains for Deep Search. One of the key resources is exposing a search across
chemistry databases like PubChem, etc and the possibility to link data with the document collections.
In this example we start with example searches for molecules both by name, synonym or SMILES.

### Access required

The content of this notebook requires access to Deep Search capabilities which are not
available on the public access system.

[Contact us](https://ds4sd.github.io) if you are interested in exploring
these Deep Search capabilities.

### Set notebook parameters

In [1]:
from dsnotebooks.settings import NotebookSettings

# notebooks settings auto-loaded from .env / env vars
notebook_settings = NotebookSettings()

PROFILE_NAME = notebook_settings.profile  # the profile to use

### Import example dependencies

In [2]:
# Import standard dependenices
from copy import deepcopy
import pandas as pd
from numerize.numerize import numerize
import mols2grid
from tqdm.notebook import tqdm

%matplotlib inline

# IPython utilities
from IPython.display import display

# Import the deepsearch-toolkit
import deepsearch as ds
from deepsearch.cps.client.components.elastic import ElasticDataCollectionSource
from deepsearch.cps.queries import DataQuery

### Connect to Deep Search

In [3]:
api = ds.CpsApi.from_env(profile_name=PROFILE_NAME)

---

## Search molecules

In this section we will look for data collections interesting for chemistry and will search for molecules

- [List data collections](#List-data-collections-in-Materials-Science-domain)
- [Search Ibuprofen on PubChem](#Search-Ibuprofen-on-PubChem)
- [Search PubChem by SMILES](#Search-PubChem-by-SMILES)
- [Search patents](#Search-SMILES-in-USPTO-patents)

### List data collections in _Materials Science_ domain

This is going to query the Deep Search system for the data collections available on the _Materials Science_ domain.
In the list we will find a combination of database and document collections.

Interesting data collections for this examples are PubChem and USTPO pre-processed patents.
Deep Search is regularly parsing the PubChem database to index molecules and their properties.

In [4]:
# Fetch list of all data collections
collections = api.elastic.list(domain="Materials Science")
collections.sort(key=lambda c: c.name.lower())

In [5]:
# Visualize summary table
results = [
    {
        "Name": c.name,
        "Type": c.metadata.type,
        "Num entries": numerize(c.documents),
        "Date": c.metadata.created.strftime("%Y-%m-%d"),
        "Coords": f"{c.source.elastic_id}/{c.source.index_key}",
    }
    for c in collections
]
display(pd.DataFrame(results))

,Name,Type,Num entries,Date,Coords
0,PubChem,Record,118.24M,2023-06-23,default/pubchem


---
## Search _Ibuprofen_ on PubChem 

In this section we search for all PubChem entries which contain the string _Ibuprofen_.

In the results table we see the name of the chemical, its molecule SMILES and some properties such as the molecular weight and the solubility.


In [6]:
# Search by name
search_query = "Ibuprofen"

data_collection = ElasticDataCollectionSource(elastic_id="default", index_key="pubchem")
page_size = 50

# Prepare the data query
query = DataQuery(
    search_query,  # The search query to be executed
    source=[
        "subject",
        "attributes",
        "identifiers",
    ],  # Which fields of documents we want to fetch
    limit=page_size,  # The size of each request page
    coordinates=data_collection,  # The data collection to be queries
)


# [Optional] Compute the number of total results matched. This can be used to monitor the pagination progress.
count_query = deepcopy(query)
count_query.paginated_task.parameters["limit"] = 0
count_results = api.queries.run(count_query)
expected_total = count_results.outputs["data_count"]
expected_pages = (
    expected_total + page_size - 1
) // page_size  # this is simply a ceiling formula


# Iterate through all results by fetching `page_size` results at the same time
all_results = []
cursor = api.queries.run_paginated_query(query)
for result_page in tqdm(cursor, total=expected_pages):
    all_results.extend(result_page.outputs["data_outputs"])

print(f"Finished fetching all data. Total is {len(all_results)} records.")

  0%|          | 0/1 [00:00<?, ?it/s]

Finished fetching all data. Total is 15 records.


In [7]:
# Parsing results. From the raw results, we will fetch
# - The CID of the PubChem record
# - The name of the chemical
# - The SMILES
# - The EC and CAS Numbers
# - The chemical and physical properties reported in PubChem, e.g. solubulity, molecular weight, etc

results_table = []
for row in all_results:
    result = {
        "cid": "",
        "chemical_name": "",
        "SMILES": "",
        "ec_number": "",
        "cas_number": "",
    }
    for ref in row["_source"]["identifiers"]:
        if ref["type"] == "cid":
            result["cid"] = ref["value"]

    for ref in row["_source"]["subject"]["identifiers"]:
        if ref["type"] == "smiles":
            result["SMILES"] = ref["value"]
        if ref["type"] == "echa_ec_number":
            result["ec_number"] = ref["value"]
        if ref["type"] == "cas_number":
            result["cas_number"] = ref["value"]

    for ref in row["_source"]["subject"]["names"]:
        if ref["type"] == "chemical_name":
            result["chemical_name"] = ref["value"]

    for attribute in row["_source"]["attributes"]:
        for predicate in attribute["predicates"]:
            value = predicate["value"]["name"]
            if "nominal_value" in predicate:
                value = predicate["nominal_value"]["value"]
            elif "numerical_value" in predicate:
                value = predicate["numerical_value"]["val"]
            result[predicate["key"]["name"]] = value

    results_table.append(result)

df = pd.DataFrame(results_table)
display(df)

,cid,chemical_name,SMILES,ec_number,cas_number,paragraph,boiling point,melting point,solubility,molecular weight,dissociation constant
0,6575,Trichloroethylene,C(=C(Cl)Cl)Cl,201-167-4,79-01-6,IDENTIFICATION AND USE: Trichloroethylene (TCE...,"1,1,2-trichloroethene has boiling point = 87.2 °C","1,1,2-trichloroethene has melting point = -99°F","1,1,2-trichloroethene has solubility = less th...",131.38,NaN
1,73981,Magnesium hydroxide,[OH-].[OH-].[Mg+2],215-170-3,1309-42-8,"LD50=8500 mg/kg (rat, oral) Common side effe...",NaN,NaN,magnesium;dihydroxide has solubility = Practic...,58.32,NaN
2,3825,Ketoprofen,CC(C1=CC(=CC=C1)C(=O)C2=CC=CC=C2)C(=O)O,244-759-8,22071-15-4,Prospective studies show that 1% to 2% of pati...,NaN,2-(3-benzoylphenyl)propanoic acid has melting ...,2-(3-benzoylphenyl)propanoic acid has solubili...,254.28,NaN
3,2662,Celecoxib,CC1=CC=C(C=C1)C2=CC(=NN2C3=CC=C(C=C3)S(=O)(=O)...,,169590-42-5,Likelihood score: B (highly likely cause of cl...,4-[5-(4-methylphenyl)-3-(trifluoromethyl)pyraz...,4-[5-(4-methylphenyl)-3-(trifluoromethyl)pyraz...,4-[5-(4-methylphenyl)-3-(trifluoromethyl)pyraz...,381.40,4-[5-(4-methylphenyl)-3-(trifluoromethyl)pyraz...
4,154699635,,CC(CC1=CC=C(C=C1)C(C)C(=O)O)CO,,,3-Hydroxyibuprofen is a known human metabolite...,NaN,NaN,NaN,222.28,NaN
5,39912,Dexibuprofen,CC(C)CC1=CC=C(C=C1)C(C)C(=O)O,,51146-56-6,Oral LD50 value in rats is 636 mg/kg.[A19261] ...,NaN,(2S)-2-[4-(2-methylpropyl)phenyl]propanoic aci...,(2S)-2-[4-(2-methylpropyl)phenyl]propanoic aci...,206.28,NaN
6,154699636,,CC(CC1=CC=C(C=C1)C(C)C(=O)O)CO,,,"R,S-3-Hydroxyibuprofen is a known human metabo...",NaN,NaN,NaN,222.28,NaN
7,5284569,Hydrocodone,CN1CCC23C4C1CC5=C2C(=C(C=C5)OC)OC3C(=O)CC4,204-733-9,125-29-1,References on the safety and potential hepatot...,NaN,"(4R,4aR,7aR,12bS)-9-methoxy-3-methyl-1,2,4,4a,...","(4R,4aR,7aR,12bS)-9-methoxy-3-methyl-1,2,4,4a,...",299.40,"(4R,4aR,7aR,12bS)-9-methoxy-3-methyl-1,2,4,4a,..."
8,1775,Phenytoin,C1=CC=C(C=C1)C2(C(=O)NC(=O)N2)C3=CC=CC=C3,211-148-2,630-93-3,Prospective studies indicate that a fairly hig...,NaN,"5,5-diphenylimidazolidine-2,4-dione has meltin...","5,5-diphenylimidazolidine-2,4-dione has solubi...",252.27,"5,5-diphenylimidazolidine-2,4-dione has logari..."
9,3672,Ibuprofen,CC(C)CC1=CC=C(C=C1)C(C)C(=O)O,239-784-6,79261-49-7 (potassium salt),The appearance of clinically apparent liver in...,2-[4-(2-methylpropyl)phenyl]propanoic acid has...,2-[4-(2-methylpropyl)phenyl]propanoic acid has...,2-[4-(2-methylpropyl)phenyl]propanoic acid has...,206.28,2-[4-(2-methylpropyl)phenyl]propanoic acid has...


### Visualize results with mols2grid

The mols2grid package is a convenient tool which visualizes all the molecules SMILES.
This section illustrates how to visualize the Deep Search results.


In [8]:
mols2grid.display(df, smiles_col="SMILES")

MolGridWidget()

---
### Search only for the chemical name 

The previous search was listing the PubChem entries which mentioned _Ibuprofen_ anywhere in their content.

Next, we will limit the search by search only in the `subject.names.value` field.

In [9]:
# Search by name
search_query = "subject.names.value:Ibuprofen"

data_collection = ElasticDataCollectionSource(elastic_id="default", index_key="pubchem")
page_size = 50

# Prepare the data query
query = DataQuery(
    search_query,  # The search query to be executed
    source=[
        "subject",
        "attributes",
        "identifiers",
    ],  # Which fields of documents we want to fetch
    limit=page_size,  # The size of each request page
    coordinates=data_collection,  # The data collection to be queries
)


# [Optional] Compute the number of total results matched. This can be used to monitor the pagination progress.
count_query = deepcopy(query)
count_query.paginated_task.parameters["limit"] = 0
count_results = api.queries.run(count_query)
expected_total = count_results.outputs["data_count"]
expected_pages = (
    expected_total + page_size - 1
) // page_size  # this is simply a ceiling formula


# Iterate through all results by fetching `page_size` results at the same time
all_results = []
cursor = api.queries.run_paginated_query(query)
for result_page in tqdm(cursor, total=expected_pages):
    all_results.extend(result_page.outputs["data_outputs"])

print(f"Finished fetching all data. Total is {len(all_results)} records.")

  0%|          | 0/1 [00:00<?, ?it/s]

Finished fetching all data. Total is 1 records.


In [10]:
# Parsing results. From the raw results, we will fetch
# - The CID of the PubChem record
# - The name of the chemical
# - The SMILES
# - The EC and CAS Numbers
# - The chemical and physical properties reported in PubChem, e.g. solubulity, molecular weight, etc

results_table = []
for row in all_results:
    result = {
        "cid": "",
        "chemical_name": "",
        "SMILES": "",
        "ec_number": "",
        "cas_number": "",
    }
    for ref in row["_source"]["identifiers"]:
        if ref["type"] == "cid":
            result["cid"] = ref["value"]

    for ref in row["_source"]["subject"]["identifiers"]:
        if ref["type"] == "smiles":
            result["SMILES"] = ref["value"]
        if ref["type"] == "echa_ec_number":
            result["ec_number"] = ref["value"]
        if ref["type"] == "cas_number":
            result["cas_number"] = ref["value"]

    for ref in row["_source"]["subject"]["names"]:
        if ref["type"] == "chemical_name":
            result["chemical_name"] = ref["value"]

    for attribute in row["_source"]["attributes"]:
        for predicate in attribute["predicates"]:
            value = predicate["value"]["name"]
            if "nominal_value" in predicate:
                value = predicate["nominal_value"]["value"]
            elif "numerical_value" in predicate:
                value = predicate["numerical_value"]["val"]
            result[predicate["key"]["name"]] = value

    results_table.append(result)


# Display the results table
df = pd.DataFrame(results_table)
display(df)

# Visualize the molecules
mols2grid.display(df, smiles_col="SMILES")

,cid,chemical_name,SMILES,ec_number,cas_number,paragraph,dissociation constant,boiling point,melting point,solubility,molecular weight
0,3672,Ibuprofen,CC(C)CC1=CC=C(C=C1)C(C)C(=O)O,239-784-6,79261-49-7 (potassium salt),The appearance of clinically apparent liver in...,2-[4-(2-methylpropyl)phenyl]propanoic acid has...,2-[4-(2-methylpropyl)phenyl]propanoic acid has...,2-[4-(2-methylpropyl)phenyl]propanoic acid has...,2-[4-(2-methylpropyl)phenyl]propanoic acid has...,206.28


MolGridWidget()

---
## Search PubChem by SMILES

In [11]:
# Search by name
search_smiles = "C1=CC=C2C(=C1)C(=CN2)CCO"
search_query = f'subject.identifiers._name:"smiles#{search_smiles.lower()}"'

data_collection = ElasticDataCollectionSource(elastic_id="default", index_key="pubchem")
page_size = 50

# Prepare the data query
query = DataQuery(
    search_query,  # The search query to be executed
    source=[
        "subject",
        "attributes",
        "identifiers",
    ],  # Which fields of documents we want to fetch
    limit=page_size,  # The size of each request page
    coordinates=data_collection,  # The data collection to be queries
)


# [Optional] Compute the number of total results matched. This can be used to monitor the pagination progress.
count_query = deepcopy(query)
count_query.paginated_task.parameters["limit"] = 0
count_results = api.queries.run(count_query)
expected_total = count_results.outputs["data_count"]
expected_pages = (
    expected_total + page_size - 1
) // page_size  # this is simply a ceiling formula


# Iterate through all results by fetching `page_size` results at the same time
all_results = []
cursor = api.queries.run_paginated_query(query)
for result_page in tqdm(cursor, total=expected_pages):
    all_results.extend(result_page.outputs["data_outputs"])

print(f"Finished fetching all data. Total is {len(all_results)} records.")

  0%|          | 0/1 [00:00<?, ?it/s]

Finished fetching all data. Total is 3 records.


In [12]:
# Parsing results. From the raw results, we will fetch
# - The CID of the PubChem record
# - The name of the chemical
# - The SMILES
# - The EC and CAS Numbers
# - The chemical and physical properties reported in PubChem, e.g. solubulity, molecular weight, etc

results_table = []
for row in all_results:
    result = {
        "cid": "",
        "chemical_name": "",
        "SMILES": "",
        "ec_number": "",
        "cas_number": "",
    }
    for ref in row["_source"]["identifiers"]:
        if ref["type"] == "cid":
            result["cid"] = ref["value"]

    for ref in row["_source"]["subject"]["identifiers"]:
        if ref["type"] == "smiles":
            result["SMILES"] = ref["value"]
        if ref["type"] == "echa_ec_number":
            result["ec_number"] = ref["value"]
        if ref["type"] == "cas_number":
            result["cas_number"] = ref["value"]

    for ref in row["_source"]["subject"]["names"]:
        if ref["type"] == "chemical_name":
            result["chemical_name"] = ref["value"]

    for attribute in row["_source"]["attributes"]:
        for predicate in attribute["predicates"]:
            value = predicate["value"]["name"]
            if "nominal_value" in predicate:
                value = predicate["nominal_value"]["value"]
            elif "numerical_value" in predicate:
                value = predicate["numerical_value"]["val"]
            result[predicate["key"]["name"]] = value

    results_table.append(result)

# Display the results table
df = pd.DataFrame(results_table)
display(df)

# Visualize the molecules
mols2grid.display(df, smiles_col="SMILES")

,cid,chemical_name,SMILES,ec_number,cas_number,molecular weight,melting point
0,11263702,,C1=CC=C2C(=C1)C(=CN2)CCO,,,166.23,NaN
1,101132237,,C1=CC=C2C(=C1)C(=CN2)CCO,,,165.22,NaN
2,10685,Tryptophol,C1=CC=C2C(=C1)C(=CN2)CCO,208-393-2,526-55-6,161.20,2-(1H-indol-3-yl)ethanol has melting point = 59°C


MolGridWidget()